# Star Trek 

In [1]:
!git clone https://github.com/jkingsman/Star-Trek-Script-Programmatics
!pip install pandas tqdm jsonlines

fatal: destination path 'Star-Trek-Script-Programmatics' already exists and is not an empty directory.
You should consider upgrading via the '/Users/ryan/github/startrek/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import json
import os
from tqdm import tqdm
from datetime import datetime as dt


In [3]:
def parse_episode(fnfn):
    ep_id = fnfn.replace('.json','').split('/processed/',1)[-1]
    series=ep_id.split('/')[0]
    ep=ep_id.split('/',1)[-1]
    o=[]
    with open(fnfn) as f:
        data=json.load(f)
        data['airdate'] = dt.fromtimestamp(data['airdate']).strftime('%Y-%m-%d')
        for k,v in data.pop('schedule').items(): data[k]=v
        for scene_i,scened in enumerate(data.pop('scenes')):
            for diald in scened.pop('dialogue'):
                text=diald.pop('line')
                od = {
                    'ep':ep_id, 
                    'scene':scene_i+1,
                    'line':len(o)+1,
                    **data,
                    **scened,
                    **diald,
                    'text':text,
                    
                }
                o.append(od)
    return o

In [4]:
# Test
pd.DataFrame(parse_episode('Star-Trek-Script-Programmatics/processed/ds9/s04e01.json'))

,ep,scene,line,airdate,stardate,title,episode,number,season,location,character,text,modifier
0,ds9/s04e01,1,1,1995-10-02,49011.4,The Way of the Warrior,1,73,4,unknown,unknown,In memory of\n Gregg Duffy Long\n Ronald W. Smith,NaN
1,ds9/s04e01,2,2,1995-10-02,49011.4,The Way of the Warrior,1,73,4,Guest quarters,O'BRIEN,O'Brien to Sisko.,voiceover
2,ds9/s04e01,2,3,1995-10-02,49011.4,The Way of the Warrior,1,73,4,Guest quarters,SISKO,Go ahead.,NaN
3,ds9/s04e01,2,4,1995-10-02,49011.4,The Way of the Warrior,1,73,4,Guest quarters,O'BRIEN,We've swept all of level seventeen. No sign of...,voiceover
4,ds9/s04e01,2,5,1995-10-02,49011.4,The Way of the Warrior,1,73,4,Guest quarters,SISKO,Move down to eighteen. We'll meet you there af...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,ds9/s04e01,70,809,1995-10-02,49011.4,The Way of the Warrior,1,73,4,Ops,ODO,Don't worry. I'm sure Dukat will take all the ...,NaN
809,ds9/s04e01,70,810,1995-10-02,49011.4,The Way of the Warrior,1,73,4,Ops,BASHIR,"Poor Garak. Dukat's a hero, and Garak's still ...",NaN
810,ds9/s04e01,70,811,1995-10-02,49011.4,The Way of the Warrior,1,73,4,Ops,DAX,"Captain, I'm receiving a priority one message ...",NaN
811,ds9/s04e01,70,812,1995-10-02,49011.4,The Way of the Warrior,1,73,4,Ops,KIRA,Looks like the Klingons are here to stay.,NaN


In [5]:
def get_all_script_filenames(searchdir='Star-Trek-Script-Programmatics/processed/'):
    o=[]
    for path,folders,files in sorted(os.walk(searchdir)):
        for fn in sorted(files):
            if fn.endswith('.json') and fn.startswith('s'):
                fnfn=os.path.join(path, fn)
                o.append(fnfn)
    return o
                

In [6]:
def iter_all_lines(files = None):
    if files is None: files = get_all_script_filenames()
    for fn in tqdm(files, desc='Iterating script files'):
        yield from parse_episode(fn)


In [7]:
def get_all_lines():
    return pd.DataFrame(iter_all_lines()).sort_values(['airdate','ep','line'])


In [8]:
df = get_all_lines()

Iterating script files: 100%|██████████| 509/509 [00:00<00:00, 1489.64it/s]


In [9]:
df.to_csv('corpus.csv', index=False)